In [ ]:
#@title environment { form-width: "10%"}
"""
ROOT/
    DATA/
        training/
        validation/
        testing/
    model/
        MODEL_NAME

!!!data file naming rule!!!
{turn}_{data_sec}_{testee}_*.txt
use "preprocess part" to split data
"""
# !pip install torchaudio==0.8.1
# !mkdir model
# !mkdir data
# !mkdir data/training
# !mkdir data/validation
# !mkdir data/testing
# !mkdir data/sundry

# !cd data/training/ && rename "s/_tien_/_5_tien_/" *.txt
# !mv data/training/*3.txt data/validation
# !mv data/training/*7.txt data/validation
# !mv data/training/*11.txt data/validation
# !rm -rf data/training/*

mv: cannot stat 'data/training/*3.txt': No such file or directory


In [ ]:
#@title hyper parameters { form-width: "10%"}
ROOT = '.'
DATA = 'data'
MODEL_NAME = 'terminator.ckpt'

WINDOW_SIZE =  500

NUM_MASK = 2
FREQ_MASK_LEN = 30

SEED = 2
batch_size = 128
learning_rate = 1e-3
num_epoch = 30


In [ ]:
#@title import { form-width: "10%"}
import os
import random
from tqdm.auto import tqdm
import numpy as np
import torch
import torch.nn as nn
import torchaudio.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchaudio.transforms as transforms

import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [ ]:
#@title preprocess { form-width: "10%"}
raw_file_path = '/content/calib_10_stop_repeat4_012_lin_2.txt'
file_num_offset = 4
round_sec = 10
data_sec = 5
trash_points = 200*20*1
file_num = 1
data_points = 200*40*3
end_points = trash_points + data_points * file_num
name = 'lin'

data = []
with open(raw_file_path, 'r') as f:
    count = 0
    for line in f.readlines():
        count += 1
        if count <= trash_points:
            continue
        try:
            arr = [float(i) for i in line.split('\t')[:5]]
        except:
            try:
                arr = arr
            except:
                arr = [float(0) for i in range(5)]
        data.append(arr)
        if count >= end_points:
            break

print(f"len(data): {len(data)}")

for i_file in range(file_num):
    for i_turn, turn in enumerate(['left', 'mid', 'right']):
        for step in range(4):
            output_file_path = f"/content/data/training/{turn}_{data_sec}_{name}_{i_file*4 + file_num_offset + step}.txt"

            with open(output_file_path, 'w') as f:
                s = i_file * data_points + (i_turn*40+step*10)*200
                print(s)
                e = s + 200 * round_sec
                for data_i in data[s:e]:
                    for value in data_i:
                        f.write(f"{value}\t")
                    f.write('\n')


In [ ]:
#@title define Dataset, functions { form-width: "10%" }

class EEG_Dataset(Dataset):
    def __init__(self, folder, mode='training'):
        self.fs = 200.0     # sample rate, Hz
        self.cutoff = 60     # cutoff frequency of the lowpass filter
        self.data = []
        self.label = []
        self.mode = mode

        files = [path for path in os.listdir(folder) if path.endswith('.txt')]
        for file_name in files:
            if not file_name.endswith('.txt'):
                print(f"{file_name} is not a data file")
                continue
            turn, data_len = file_name.split('_')[:2]
            try:
                data_len = int(data_len)*200
            except:
                data_len = 123
            count = 1
            voltage = []
            alpha = []
            beta = []
            delta = []
            theta = []
            with open(os.path.join(folder, file_name), 'r') as f:
                for line in f.readlines():
                    try:
                        arr = [float(i) for i in line.split('\t')[:5]]
                    except:
                        try:
                            arr = arr
                        except:
                            arr = [float(0) for i in range(5)]
                    voltage.append(arr[0])
                    alpha.append(arr[1])
                    beta.append(arr[2])
                    delta.append(arr[3])
                    theta.append(arr[4])
                    if (count >= data_len and mode != 'testing'):
                        break
                    count += 1

            voltage = F.lowpass_biquad(waveform=torch.tensor(voltage), sample_rate=self.fs, cutoff_freq=self.cutoff)
            alpha = torch.tensor(alpha)
            beta = torch.tensor(beta)
            delta = torch.tensor(delta)
            theta = torch.tensor(theta)

            label = -1
            if mode == 'training' or mode == 'validation':
                data_i = torch.stack((voltage, alpha, beta, delta, theta))
                if turn == 'mid':
                    label = 1
                elif turn == 'left':
                    label = 0
                elif turn == 'right':
                    label = 2
                else:
                    raise NameError('Illegal data file name')
                self.data.append(data_i)
                self.label.append(label)
            elif mode == 'testing':
                data_i = torch.stack((voltage, alpha, beta, delta, theta))
                self.data.append(data_i)
                self.label.append(-1)
            else:
                raise NameError('Illegal mode')
        
        if mode == 'testing':
            self.data_len = len(voltage)
        else:
            self.data_len = data_len

  
    def __getitem__(self, index):
        window_offset = self.data_len - WINDOW_SIZE
        data_idx = int(index/window_offset)
        start = index % window_offset
        end = start + WINDOW_SIZE

        window_v = self.data[data_idx][0][start : end]
        window_alpha = self.data[data_idx][1][start : end]
        window_beta = self.data[data_idx][2][start : end]
        window_delta = self.data[data_idx][3][start : end]
        window_theta = self.data[data_idx][4][start : end]

        sp = torch.fft.fft(window_v)
        # sp = torch.fft.rfft(window_v)

        sp_real = sp.real
        sp_imag = sp.imag
        if self.mode == 'training':
            for _ in range(NUM_MASK):
                sp_real = transforms.FrequencyMasking(FREQ_MASK_LEN)(sp_real.view(-1))
                sp_imag = transforms.FrequencyMasking(FREQ_MASK_LEN)(sp_imag.view(-1))
                window_alpha = transforms.FrequencyMasking(FREQ_MASK_LEN)(window_alpha.view(-1))
                window_beta = transforms.FrequencyMasking(FREQ_MASK_LEN)(window_beta.view(-1))
                window_delta = transforms.FrequencyMasking(FREQ_MASK_LEN)(window_delta.view(-1))
                window_theta = transforms.FrequencyMasking(FREQ_MASK_LEN)(window_theta.view(-1))

        data_i = torch.stack((sp_real.view(-1), sp_imag.view(-1), window_alpha.view(-1), window_beta.view(-1), window_delta.view(-1), window_theta.view(-1)))
        if self.mode == 'testing':
            label_i = torch.tensor(-1, dtype=torch.long)
        else:
            label_i = torch.tensor(self.label[data_idx], dtype=torch.long)
        return data_i, label_i
    
    def __len__(self):
        return len(self.data) * (self.data_len - WINDOW_SIZE)


In [ ]:
#@title define model { form-width: "10%" }

def drop_connect(inputs, p, training):
    if not training:
        return inputs
    batch_size = inputs.shape[0]
    keep_prob = 1 - p
    random_tensor = keep_prob
    random_tensor += torch.rand([batch_size, 1, 1], dtype=inputs.dtype, device=inputs.device)

    binary_tensor = torch.floor(random_tensor) 
    output = inputs / keep_prob * binary_tensor
    return output

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.large = False
        self.channel = [6, 12, 16, 32, 16]
        self.probs = [0.1, 0.2, 0.3, 0.4]

        self.maxPool = nn.MaxPool1d(2)

        self.cnn0 = nn.Sequential(
            nn.Conv1d(self.channel[0], self.channel[0], 3, 1, 1),
            nn.BatchNorm1d(self.channel[0]),
            nn.ReLU(),
        )
        self.p0 = nn.Sequential(
            nn.Conv1d(self.channel[0], self.channel[1], 3, 1, 1),
            nn.BatchNorm1d(self.channel[1]),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.cnn1 = nn.Sequential(
            nn.Conv1d(self.channel[1], self.channel[1], 3, 1, 1),
            nn.BatchNorm1d(self.channel[1]),
            nn.ReLU(),
        )
        self.p1 = nn.Sequential(
            nn.Conv1d(self.channel[1], self.channel[2], 3, 1, 1),
            nn.BatchNorm1d(self.channel[2]),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.cnn2 = nn.Sequential(
            nn.Conv1d(self.channel[2], self.channel[2], 3, 1, 1),
            nn.BatchNorm1d(self.channel[2]),
            nn.ReLU(),
        )
        self.p2 = nn.Sequential(
            nn.Conv1d(self.channel[2], self.channel[3], 3, 1, 1),
            nn.BatchNorm1d(self.channel[3]),
            # nn.ReLU(),
        )

        # self.cnn3 = nn.Sequential(
        #     nn.Conv1d(self.channel[3], self.channel[3], 3, 1, 1),
        #     nn.BatchNorm1d(self.channel[3]),
        #     nn.ReLU(),
        # )
        # self.p3 = nn.Sequential(
        #     nn.Conv1d(self.channel[3], self.channel[4], 3, 1, 1),
        #     nn.BatchNorm1d(self.channel[4]),
        #     nn.ReLU(),
        # )

        self.adaptive_avg_pool = nn.AdaptiveAvgPool1d(1)

        self.fc2 = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.channel[3], 3),
        )

    def forward(self, x):
        x = x.view(x.size()[0], 6, -1)
        y = self.cnn0(x)
        y = drop_connect(y, self.probs[0], self.training)
        x = self.p0(x + y)

        y = self.cnn1(x)
        y = drop_connect(y, self.probs[1], self.training)
        x = self.p1(x + y)

        y = self.cnn2(x)
        y = drop_connect(y, self.probs[2], self.training)
        x = self.p2(x + y)

        # y = self.cnn3(x)
        # y = drop_connect(y, self.probs[3], self.training)
        # x = self.p3(x + y)

        y = self.adaptive_avg_pool(x)
        y = y.view(y.size()[0], -1)
        out = self.fc2(y)
        return out, y


In [ ]:
#@title Prepare { form-width: "10%" }
# 200 points per sec

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

train_set = EEG_Dataset(folder='data/training', mode='training')
valid_set = EEG_Dataset(folder='data/validation', mode='validation')

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

model = Classifier().cuda()
loss = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,threshold=1e-4,patience=7,verbose=True,min_lr=2e-6)


In [ ]:
#@title Training { form-width: "10%" }

now_best_loss = 123
for epoch in range(num_epoch):
    train_accs = 0.0
    train_loss = 0.0
    valid_accs = 0.0
    valid_loss = 0.0

    model.train()

    for batch in tqdm(train_loader):
        data, label = batch
        optimizer.zero_grad()
        pred, y = model(data.cuda())
        loss_bt = loss(pred, label.cuda())
        loss_bt.backward()
        optimizer.step()

        train_accs += np.sum(np.argmax(pred.cpu().data.numpy(), axis=1) == label.numpy())
        train_loss += loss_bt.item()*len(label)
    train_acc = train_accs/len(train_set)
    train_loss = train_loss/len(train_set)

    # print(f"[{epoch+1:2d}/{num_epoch:2d}], train_acc={train_acc:.3f}\t train_loss={train_loss:.3f}")
    # continue
    model.eval()
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            data, label = batch
            pred, y = model(data.cuda())
            loss_bt = loss(pred, label.cuda())

            valid_accs += np.sum(np.argmax(pred.cpu().data.numpy(), axis=1) == label.numpy())
            valid_loss += loss_bt.item()*len(label)
        valid_acc = valid_accs/len(valid_set)
        valid_loss = valid_loss/len(valid_set)
        if valid_loss < now_best_loss:
            now_best_loss = valid_loss
            torch.save(model, os.path.join(ROOT, 'model', MODEL_NAME))
            print('model saved')
        scheduler.step(valid_loss)

    print(f"[{epoch+1:2d}/{num_epoch:2d}], train_acc={train_acc:.3f}\t train_loss={train_loss:.3f}, \
                valid_acc={valid_acc:.3f}\t valid_loss={valid_loss:.3f}")


In [ ]:
#@title load model and do validation { form-width: "10%" }
model = torch.load(os.path.join(ROOT, 'model', MODEL_NAME))
model.eval()
valid_accs = 0
valid_loss = 0
with torch.no_grad():
    for batch in tqdm(valid_loader):
        data, label = batch
        pred, _ = model(data.cuda())
        loss_bt = loss(pred, label.cuda())
        valid_accs += np.sum(np.argmax(pred.cpu().data.numpy(), axis=1) == label.numpy())
        valid_loss += loss_bt.item()*len(label)
    valid_acc = valid_accs/len(valid_set)
    valid_loss = valid_loss/len(valid_set)
print(f"valid_acc {valid_acc},  valid_loss {valid_loss}")

In [ ]:
#@title Testing { form-width: "10%" }
# model = Classifier().cuda()


test_file ='/content/data/testing/test_new_lin_1.txt'
# tune_epoch = 10
vote_range = 256
test_step = 20

# model = Classifier().cuda()
# model = torch.load(os.path.join(ROOT, 'model', MODEL_NAME))

test_set = EEG_Dataset(folder='data/testing', mode='testing')
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

model.eval()
prediction = []
with torch.no_grad():
    for batch in test_loader:
        data, _ = batch
        pred, _ = model(data.cuda())
        prediction.extend(np.argmax(pred.cpu().data.numpy(), axis=1))

prediction_voted = []
for j in range(vote_range, len(prediction), test_step):
    candidate = prediction[j-vote_range : j]
    # p = max(set(candidate), key = candidate.count)
    v = [0, 0, 0]
    for c in candidate:
        v[c] += 1
    if max(v) >= vote_range*0.7:
        p = v.index(max(v))
    else:
        p = 1
    prediction_voted.append(p)

print(prediction)
print(prediction_voted)
print(len(prediction))
print(f"prediction: {len(prediction_voted)/10} secs")

with open(f"predict.txt", 'w') as f:
    for p in prediction_voted:
        f.write(f"{str(p)},")
    f.write('\n')


[2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 0, 0, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 

In [ ]:
#@title plot { form-width: "10%" }
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

# train_data = EEG_Dataset(root=root_path, mode='training')
# data = [np.array(d) for l, d in train_data]
# label = [int(l) for l, d in train_data]

data = []
label = []
for i, batch in enumerate(valid_loader):
    pred, emb = model(batch[1].cuda())
    d_emb = emb.detach().cpu()
    for d in d_emb:
        data.append(np.array(d))
    for l in batch[0]:
        label.append(int(l))


X = np.array(data)
X_embedded = TSNE(n_components=2).fit_transform(X)
X_embedded.shape


x_min, x_max = X_embedded.min(0), X_embedded.max(0)
X_norm = (X_embedded - x_min) / (x_max - x_min)
plt.figure(figsize=(8, 8))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], str(label[i]), color=plt.cm.Set1(label[i]), 
             fontdict={'weight': 'bold', 'size': 9})
plt.xticks([])
plt.yticks([])
plt.show()